In [1]:
import arviz as az
import numpy as np
import pandas as pd
import pymc as pm
from pymc.math import exp, ge, switch

In [2]:
data = pd.read_csv("bladderc.csv")

In [3]:
data.head(20)

,time,observed,group
0,0,0,0
1,1,0,0
2,4,0,0
3,7,0,0
4,10,0,0
5,6,1,0
6,14,0,0
7,18,0,0
8,5,1,0
9,12,1,0


So if we oberved the tumor, we want to right-censor the data. If not, we don't.

In [6]:
data["censored"] = 0.0
data.loc[data["observed"] == 0, "censored"] = data["time"]
data.loc[data["observed"] == 1, "censored"] = data["time"] + 1

y = data["time"].to_numpy(copy=True)
x = data["group"].to_numpy(copy=True)
censored = data["censored"].to_numpy(copy=True)

array([ 0,  1,  4,  7, 10,  6, 14, 18,  5, 12, 23, 10,  3,  3,  7,  3, 26,
        1,  2, 25, 29, 29, 29, 28,  2,  3, 12, 32, 34, 36, 29, 37,  9, 16,
       41,  3,  6,  3,  9, 18, 49, 35, 17,  3, 59,  2,  5,  2,  1,  1,  5,
        9, 10, 13,  3,  1, 18, 17,  2, 17, 22, 25, 25, 25,  6,  6,  2, 26,
       38, 22,  4, 24, 41, 41,  1, 44,  2, 45,  2, 46, 49, 50,  4, 54, 38,
       59])

In [ ]:
# right-censored model
#inits = {'beta0': np.array(.1), 'beta1': np.array(.1)}

with pm.Model() as m:
    beta0 = pm.Normal("beta0", 0, tau=0.0001)
    beta1 = pm.Normal("beta1", 0, tau=0.0001)
    λ = exp(beta0 + beta1 * x)
    obs_latent = pm.Exponential.dist(lam=λ)
    likelihood = pm.Censored(
        "likelihood", obs_latent, lower=None, upper=censored, observed=y
    )

    mu0 = pm.Deterministic("mu0", exp(-beta0))
    mu1 = pm.Deterministic("mu1", exp(-beta0 - beta1))

    mu_diff = pm.Deterministic("mu_diff", mu1 - mu0)

    H_prob = pm.Deterministic("H_prob", switch(ge(mu_diff, 0), 1, 0))

    trace = pm.sample(
        2000,  # samples
        chains=4,
        tune=500,
        cores=4,
        init="jitter+adapt_diag",
        random_seed=1,
        return_inferencedata=True,
    )

az.summary(trace, hdi_prob=0.9)

Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta0, beta1]


/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/step_methods/hmc/quadpotential.py:258: RuntimeWarning: divide by zero encountered in true_divide
  np.divide(1, self._stds, out=self._inv_stds)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/step_methods/hmc/quadpotential.py:237: RuntimeWarning: invalid value encountered in multiply
  return np.multiply(self._var, x, out=out)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/step_methods/hmc/quadpotential.py:258: RuntimeWarning: divide by zero encountered in true_divide
  np.divide(1, self._stds, out=self._inv_stds)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/step_methods/hmc/quadpotential.py:237: RuntimeWarning: invalid value encountered in multiply
  return np.multiply(self._var, x, out=out)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/step_methods/hmc/quadpotential.py:258: RuntimeWarning: div

In [ ]:
# testing
#inits = {'beta0': np.array(.1), 'beta1': np.array(.1)}

data = pd.read_csv("bladderc.csv")

data["censored"] = 0.0
data.loc[data["observed"] == 0, "censored"] = data["time"]
data.loc[data["observed"] == 1, "censored"] = np.inf

y = data["time"].to_numpy(copy=True)
x = data["group"].to_numpy(copy=True)
censored = data["censored"].to_numpy(copy=True)


with pm.Model() as m:
    x_data = pm.Data("x_data", x, mutable=True)
    y_data = pm.Data("y_data", y, mutable=False)
    censored_data = pm.Data("censored_data", censored, mutable=True)

    beta0 = pm.Normal("beta0", 0, sigma=10)
    beta1 = pm.Normal("beta1", 0, sigma=10)
    λ = exp(beta0 + beta1 * x_data)
    obs_latent = pm.Exponential.dist(lam=λ)
    likelihood = pm.Censored(
        "likelihood", obs_latent, lower=None, upper=censored, observed=y_data
    )

    mu0 = pm.Deterministic("mu0", exp(-beta0))
    mu1 = pm.Deterministic("mu1", exp(-beta0 - beta1))

    mu_diff = pm.Deterministic("mu_diff", mu1 - mu0)

    H_prob = pm.Deterministic("H_prob", switch(ge(mu_diff, 0), 1, 0))

    trace = pm.sample(
        2000,  # samples
        chains=4,
        tune=500,
        init="adapt_diag",
        random_seed=1,
        return_inferencedata=True,
    )

az.summary(trace, hdi_prob=0.9)

Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta0, beta1]


/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/step_methods/hmc/quadpotential.py:258: RuntimeWarning: divide by zero encountered in true_divide
  np.divide(1, self._stds, out=self._inv_stds)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/step_methods/hmc/quadpotential.py:237: RuntimeWarning: invalid value encountered in multiply
  return np.multiply(self._var, x, out=out)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/step_methods/hmc/quadpotential.py:258: RuntimeWarning: divide by zero encountered in true_divide
  np.divide(1, self._stds, out=self._inv_stds)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/step_methods/hmc/quadpotential.py:237: RuntimeWarning: invalid value encountered in multiply
  return np.multiply(self._var, x, out=out)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/step_methods/hmc/quadpotential.py:258: RuntimeWarning: div

In [7]:
az.summary(trace, hdi_prob=0.9)

NameError: name 'trace' is not defined

ERROR! Session/line number was not unique in database. History logging moved to new session 566
